In [1]:


import sys 
print(sys.version)


3.9.13 (v3.9.13:6de2ca5339, May 17 2022, 11:37:23) 
[Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import re

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

import string
nltk.download('wordnet') # first-time use only
nltk.download('omw-1.4')
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pometry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pometry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/pometry/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Getting Sentiment

In [3]:
INPUT = "/Users/pometry/Desktop/Features/sentiment.csv"
OUTPUT_SENTIMENT ="sentiment.csv"
OUTPUT_LABELS = "nlp_labelled.csv"
MAX_SEQUENCE_LENGTH = 30

In [4]:
model = load_model("best_model.hdf5")
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [5]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [6]:
def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [8]:
def decode_sentiment(score):
    if score>0.6:
        return "Positive"
    elif score<0.4:
        return "Negative"
    else:
        return "Neutral"

In [9]:
data = pd.read_csv(INPUT)
# data.drop(columns=data.columns[0], axis=1, inplace=True)
data.dropna(subset=["author_id", "text"], inplace=True)
data

,author_id,conversation_id,created_at,entities,id,in_reply_to_user_id,lang,public_metrics,referenced_tweets,source,text,label,sentiment_score,sentiment
0,398049962,1.309378e+18,1.601015e+09,"{'hashtags': [{'text': 'BTC', 'indices': [28, ...",1.309378e+18,NaN,en,"{'retweet_count': 840, 'reply_count': None, 'l...",NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @Bybit_Official: Bybit's #BTC Draw 🎉\n\nTha...,human,0.654118,Positive
1,3308017024,1.309379e+18,1.601015e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.309379e+18,42623331.0,en,"{'retweet_count': 0, 'reply_count': None, 'lik...",NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@rfleschenberg What was the bug root cause?,human,0.545559,Neutral
2,440108963,1.309386e+18,1.601017e+09,"{'hashtags': [{'text': 'Acquirers', 'indices':...",1.309386e+18,NaN,en,"{'retweet_count': 0, 'reply_count': None, 'lik...",NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",How #Acquirers and #Merchants Can Beat the Odd...,human,0.556009,Neutral
3,1242334305948274688,1.309388e+18,1.601018e+09,"{'hashtags': [{'text': 'telehealth', 'indices'...",1.309388e+18,NaN,en,"{'retweet_count': 0, 'reply_count': None, 'lik...",NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Looking for a trustworthy Telehealth App?\nWe ...,human,0.251360,Negative
4,19294743,1.309392e+18,1.601018e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.309392e+18,NaN,en,"{'retweet_count': 0, 'reply_count': None, 'lik...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",It was very interesting to explore the impact ...,human,0.963745,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076547,1391848122217734146,1.503399e+18,1.647273e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.503399e+18,NaN,en,"{'retweet_count': 1, 'reply_count': None, 'lik...",NaN,"<a href=""http://twitter.com/download/android"" ...",RT @Jason_R_Gillis: @DanielVanDam6 @ElFinancie...,human,0.385671,Negative
2076548,801717287300530176,1.503399e+18,1.647273e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.503399e+18,NaN,en,"{'retweet_count': 1, 'reply_count': None, 'lik...",NaN,"<a href=""http://twitter.com/download/android"" ...",RT @pzwarne: @mjtward19 Putin's told him to ke...,human,0.798272,Positive
2076549,1391848122217734146,1.503399e+18,1.647273e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.503399e+18,NaN,en,"{'retweet_count': 1, 'reply_count': None, 'lik...",NaN,"<a href=""http://twitter.com/download/android"" ...",RT @Jason_R_Gillis: @MGH1955 @ElFinanciero_Mx ...,human,0.429379,Neutral
2076550,1391848122217734146,1.503399e+18,1.647273e+09,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.503399e+18,NaN,en,"{'retweet_count': 1, 'reply_count': None, 'lik...",NaN,"<a href=""http://twitter.com/download/android"" ...",RT @Jason_R_Gillis: @MalthusGamba @ElFinancier...,human,0.273655,Negative


In [10]:
puretext = data[["author_id", "text"]].copy()
puretext.dropna(inplace=True)
puretext

,author_id,text
0,398049962,RT @Bybit_Official: Bybit's #BTC Draw 🎉\n\nTha...
1,3308017024,@rfleschenberg What was the bug root cause?
2,440108963,How #Acquirers and #Merchants Can Beat the Odd...
3,1242334305948274688,Looking for a trustworthy Telehealth App?\nWe ...
4,19294743,It was very interesting to explore the impact ...
...,...,...
2076547,1391848122217734146,RT @Jason_R_Gillis: @DanielVanDam6 @ElFinancie...
2076548,801717287300530176,RT @pzwarne: @mjtward19 Putin's told him to ke...
2076549,1391848122217734146,RT @Jason_R_Gillis: @MGH1955 @ElFinanciero_Mx ...
2076550,1391848122217734146,RT @Jason_R_Gillis: @MalthusGamba @ElFinancier...


In [11]:
puretext.text = puretext.text.apply(lambda x: preprocess(x))

In [ ]:
x = pad_sequences(tokenizer.texts_to_sequences(puretext.text), maxlen = MAX_SEQUENCE_LENGTH)

In [ ]:
scores = model.predict(x, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

In [ ]:
data["sentiment_score"] = scores
data["sentiment"] = y_pred_1d
data

In [ ]:
data.to_csv(OUTPUT_SENTIMENT, index=False)

# Getting Tweet Similarity and Lexical Diversity

In [12]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [13]:
nltk.download('punkt')
train_text = pd.read_csv("train_text.csv")
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
TfidfVec.fit(train_text)

[nltk_data] Downloading package punkt to /Users/pometry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


TfidfVectorizer(stop_words='english',
                tokenizer=<function LemNormalize at 0x360a374c0>)

In [14]:
def cos_similarity(textlist):
    if textlist == []:
        return 0
    else:
        tfidf = TfidfVec.transform(textlist)
        return (tfidf * tfidf.T).toarray()

In [15]:
def token_type_ratio(string):
    string = re.sub(r'[^\w]', ' ', string)
    string = string.lower()
    tokens = nltk.word_tokenize(string)
    types = nltk.Counter(tokens)
    if tokens == []:
        return 0
    else:
        return (len(types)/len(tokens))*100

In [16]:
def average(array):
    average = []
    for i in range(len(array)):
        for j in range(len(array)):
            if i != j:
                average.append(j)
    if len(average) == 0:
        return 0
    else:
        return sum(average)/len(average)

In [17]:
users = puretext["author_id"].unique().tolist()

In [18]:
sim_dict = {}
count=0
for a_id in tqdm(users):
    if(count <=126000):
        tweets = puretext[puretext["author_id"]==a_id].text.tolist()
        sim = cos_similarity(tweets)
        sim_dict[a_id] = average(sim)
        count+=1
df_sim = pd.DataFrame(sim_dict.items(), columns=["author_id", "tweet_similarity"])

100%|██████████████████████████████████| 142285/142285 [06:47<00:00, 348.88it/s]


In [20]:
ttr_dict = {}

count=0
for a_id in tqdm(users):
    if(count <=126000):
        count+=1
        tweets = puretext[puretext["author_id"]==a_id].text.tolist()
        ttrs = [token_type_ratio(tweet) for tweet in tweets]
        if ttrs == []:
            ttr_dic[a_id] = 0
        else:
            ttr_dict[a_id] = sum(ttrs)/len(ttrs)
df_ttr = pd.DataFrame(ttr_dict.items(), columns=["author_id", "lexical_diversity"])

100%|██████████████████████████████████| 142285/142285 [04:33<00:00, 519.44it/s]


In [21]:
df_sent = data.groupby("author_id")["sentiment_score"].mean().to_frame()
df_sent.reset_index(inplace=True)
df_sent.columns = ["author_id","sentiment"]

In [22]:
new_data = pd.merge(df_sim, df_ttr, on="author_id")
new_data

,author_id,tweet_similarity,lexical_diversity
0,398049962,16.0,98.070940
1,3308017024,8.0,98.403361
2,440108963,5.5,99.691358
3,1242334305948274688,16.0,96.645919
4,19294743,12.0,99.636364
...,...,...,...
125996,1377218434380013570,0.0,100.000000
125997,524200177,3.5,96.964286
125998,1202945010,1.0,95.833333
125999,1101855900959006720,0.0,100.000000


In [23]:
new_data = pd.merge(new_data, df_sent, on="author_id")
new_data

,author_id,tweet_similarity,lexical_diversity,sentiment
0,398049962,16.0,98.070940,0.644564
1,3308017024,8.0,98.403361,0.737589
2,440108963,5.5,99.691358,0.634190
3,1242334305948274688,16.0,96.645919,0.596558
4,19294743,12.0,99.636364,0.646406
...,...,...,...,...
125996,1377218434380013570,0.0,100.000000,0.756398
125997,524200177,3.5,96.964286,0.597547
125998,1202945010,1.0,95.833333,0.827006
125999,1101855900959006720,0.0,100.000000,0.361859


In [24]:
new_data.to_csv(OUTPUT_LABELS, index=False)

# Time Between Tweets

In [ ]:
time_data = data.sort_values(["author_id", "created_at"],ascending=True).groupby("author_id")
time_data["diff"] =  time_data["created_at"] - time_data["created_at"].shift()
time_data = time_data.sort_values(["author_id", "created_at"])
time_data["match"] = time_data.author_id.eq(time_data.author_id.shift())
time_data.loc[time_data["match"] == False , "diff"] = None